<a href="https://colab.research.google.com/github/Evil-Frog/Sequence-Models-coursera/blob/master/Snake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import os
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
# Detect hardware, return appropriate distribution strategy
IMAGE_SIZE = [128,128]
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:           
    strategy = tf.distribute.get_strategy() # default distribution strategy in Tensorflow. Works on CPU and single GPU.

print("REPLICAS: ", strategy.num_replicas_in_sync)

REPLICAS:  1


In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d slimee/snakedataset

 84% 81.0M/95.9M [00:00<00:00, 61.1MB/s]
100% 95.9M/95.9M [00:01<00:00, 94.3MB/s]


In [ ]:
from zipfile import ZipFile
file_name = "snakedataset.zip"
with ZipFile(file_name,'r') as zip:
    zip.extractall()
    print("Done")

Done


In [ ]:
DATA_DIR = '/content/dataset'
train_df = pd.read_csv(DATA_DIR+'/train.csv')
test_df = pd.read_csv(DATA_DIR+'/test.csv')
train_df.sort_values(by=['image_id'],inplace=True)
train_df_array = train_df.to_numpy()
labels = pd.get_dummies(train_df.breed)
labels_one_hot = labels.to_numpy()
print(labels_one_hot.shape)
labels= tf.convert_to_tensor(labels_one_hot[:,1])
train_label = train_df_array[:,1]
print(test_df.size)

(5508, 35)
2361


In [ ]:
print(test_df.iloc[3,0])

1f36f26994


In [ ]:
train_data_img = []
train_data_label = []
train = []
index = 0
new_size = 128
train_dir = DATA_DIR+'/train/'

!ls
for filename in sorted( os.listdir(DATA_DIR+'/train')):    
    try:
        t_img=cv2.imread(os.path.join(train_dir,filename),cv2.IMREAD_COLOR)
        t_img_resize = cv2.resize(t_img,(new_size,new_size))
        t_img_resize = np.array(t_img_resize)
        train_data_label.append(labels_one_hot[index])
        train_data_img.append(t_img_resize)
    except Exception as e :
        pass
    index+=1
train_data_img = np.array(train_data_img)
test_dir = DATA_DIR + '/test/'
#for filename in test_df['image_id']:
    #fileName = test_df[filename]
    #fileNameValue = fileName.values + '.jpg'
    #print(fileNameValue)
    #try:
    #    testing_img = cv2.imread(os.path.join(test_dir,fileNameValue))
    #    testing_img_resize = cv2.resize(testing_img,(new_size,new_size))
    #    testing_img_resize = np.array(testing_img_resize)
    #except Exception as e :
    #    pass
index = 0 
test_data_img = []
while index < test_df.size:
    filename = test_df.iloc[index,0]
    filename = filename + '.jpg'
    test_image = cv2.imread(os.path.join(test_dir,filename),cv2.IMREAD_COLOR)
    test_image_resize = cv2.resize(test_image,(new_size,new_size))
    test_image_resize = np.array(test_image_resize)
    test_data_img.append(test_image_resize)
    index += 1
test_data_image = np.array(test_data_img)
print(train_data_img.shape)

dataset  kaggle.json  sample_data  snakedataset.zip
(5508, 128, 128, 3)


In [ ]:
with strategy.scope():
    pretrained_model = tf.keras.applications.DenseNet201(weights='imagenet', include_top=False ,input_shape=[*IMAGE_SIZE, 3])
    #pretrained_model = tf.keras.applications.Xception(weights='imagenet', include_top=False ,input_shape=[*IMAGE_SIZE, 3])
    #pretrained_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False ,input_shape=[*IMAGE_SIZE, 3])
    #pretrained_model=tf.keras.applications.InceptionResNetV2(weights='imagenet',include_top=False,input_shape=[*IMAGE_SIZE,3])
    pretrained_model.trainable = False # False = transfer learning, True = fine-tuning
    
    model = tf.keras.Sequential([
        pretrained_model,
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(35,activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Softmax(),
    ])
        
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
    loss = 'sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy']
)
model.summary()

74842112/74836368 [==============================] - 1s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet201 (Functional)     (None, 4, 4, 1920)        18321984  
_________________________________________________________________
flatten (Flatten)            (None, 30720)             0         
_________________________________________________________________
dropout (Dropout)            (None, 30720)             0         
_________________________________________________________________
batch_normalization (BatchNo (None, 30720)             122880    
_________________________________________________________________
dense (Dense)                (None, 35)                1075235   
_________________________________________________________________
batch_normalization_1 (Batch (None, 35)                140       
_________________________________________________________

In [ ]:
train_data_img = tf.convert_to_tensor(train_data_img,dtype=np.float32)
test_data_image = tf.convert_to_tensor(test_data_image,dtype=np.float32)
model.fit(train_data_img,labels,batch_size=64,epochs=10,validation_split=0.2)

Epoch 1/10
69/69 [==============================] - 13s 181ms/step - loss: 0.7619 - sparse_categorical_accuracy: 0.9051 - val_loss: 0.1209 - val_sparse_categorical_accuracy: 0.9746
Epoch 2/10
69/69 [==============================] - 8s 113ms/step - loss: 0.1409 - sparse_categorical_accuracy: 0.9730 - val_loss: 0.1157 - val_sparse_categorical_accuracy: 0.9746
Epoch 3/10
69/69 [==============================] - 8s 114ms/step - loss: 0.1258 - sparse_categorical_accuracy: 0.9730 - val_loss: 0.1155 - val_sparse_categorical_accuracy: 0.9746
Epoch 4/10
69/69 [==============================] - 8s 115ms/step - loss: 0.1104 - sparse_categorical_accuracy: 0.9734 - val_loss: 0.1194 - val_sparse_categorical_accuracy: 0.9746
Epoch 5/10
69/69 [==============================] - 8s 115ms/step - loss: 0.0940 - sparse_categorical_accuracy: 0.9737 - val_loss: 0.1431 - val_sparse_categorical_accuracy: 0.9691
Epoch 6/10
69/69 [==============================] - 8s 116ms/step - loss: 0.0853 - sparse_categoric

In [ ]:
#model.save("snake_classifier.h5")
#files.download("snake_classifier.h5")

In [ ]:
#files.upload()

In [ ]:
res=model.predict(test_data_image)
#res=model.predict_classes(test_data_image)
#for re in res:
    #print(res)

tf.Tensor(
[   7 2293 1464 1464 1464 1464 1464 1464 1464 1464 1464 1464 1464 1464
 1464 1464 1464 1464 1464 1464 1464 1464 1464 1464 1464 1464 1464 1464
 1464 1464 1464 1464 1464 1464 1464], shape=(35,), dtype=int64)
